#### CSCE 670 :: Information Storage and Retrieval :: Texas A&M University :: Spring 2017


# Homework 1:  Retrieval Models: Boolean + Vector Space

### 100 points [5% of your final grade]

### Due: Thursday, February 2 by 11:59pm

*Goals of this homework:* In this homework you will get first hand experience building a text-based mini search engine. In particular, there are three main learning objectives: (i) the basics of tokenization (e.g. stemming, case-folding, etc.) and its effect on information retrieval; (ii) basics of index building and Boolean retrieval; and (iii) basics of the Vector Space model and ranked retrieval.

*Submission Instructions:* To submit your homework, rename this notebook as lastname_firstinitial_hw#.ipynb. For example, my homework submission would be: caverlee_j_hw1.ipynb. Submit this notebook via ecampus. Your notebook should be completely self-contained, with the results visible in the notebook. 

*Late submission policy:* For this homework, you may use up to three of your late days, meaning that no submissions will be accepted after Friday, February 5 at 11:59pm.

## Dataset

We provide the dataset, [southpark_scripts.zip](https://www.dropbox.com/s/6rzfsbn97s8vwof/southpark_scripts.zip), which includes scripts for episodes of the first twenty seasons of the TV show South Park. You will build an inverted index over these scripts where each episode should be treated as a single document. There are 277 episodes (documents) to index and search on.

## (20 points) Part 1: Parsing

First, you should tokenize documents using **whitespaces and punctuations as delimiters** but do not remove stop words. Your parser needs to also provide the following two confgiuration options:
* Case-folding
* Stemming: use [nltk Porter stemmer](http://www.nltk.org/api/nltk.stem.html#module-nltk.stem.porter)

Please note that you should stick to the stemming package listed above. Otherwise, given the same query, the results generated by your code can be different from others.

In [28]:
# configuration options
use_stemming = True # or false
use_casefolding = True # or false

In [11]:
# Your parser function here. It will take the two option variables above as the parameters
# add cells as needed to organize your code

def parser(use_stemming, use_casefolding):
    import glob
    import re
    from nltk import stem
    
    documents=[]
    names=[]
    stemmer = stem.PorterStemmer()
    rs=[];
    for filename in glob.glob('*.txt'):
        names.append(filename)
        fl=open(filename)
        pre_words=fl.read();
        words=pre_words.decode("utf8",'ignore')   #maybe something wrong here to ignore.
        #word_list=re.split(";\W*|,\W*|\*\W*|\n\W*|\s+|-\W*|\.\W*|\:\W*|\?\W*|!\W*|_\W*|\'\W*",words)
        word_list=re.split("\W+",words);
        stemmed=[]
        case_fold=[]
        origin=[]
        for word in word_list:
            if not word:
                continue
            origin.append(word)
            case_fold.append(word.lower())
            if use_casefolding:
                stemmed.append(stemmer.stem(word.lower()))
            else:
                stemmed.append(stemmer.stem(word))
        if use_stemming:
            documents.append(stemmed)
        elif use_casefolding:
            documents.append(case_fold)
        else:
            documents.append(origin)
    return [documents, names];

res=parser(use_stemming, use_casefolding)
documents=res[0];
names=res[1];

### Observations

Once you have your parser working, you should report here the size of your dictionary under the four cases. That is, how many unique tokens do you have with stemming on and casefolding on? And so on. You should fill in the following

* Stemming + Casefolding       = 17143
* Stemming + No Casefolding    = 22221
* No Stemming + Casefolding    = 23806
* No Stemming + No Casefolding = 29550


## (40 points) Part 2: Boolean Retrieval

In this part you build an inverted index to support Boolean retrieval. We only require your index to  support AND queries. In other words, your index does not have to support OR, NOT, or parentheses. Also, we do not explicitly expect to see AND in queries, e.g., when we query **great again**, your search engine should treat it as **great** AND **again**.

Example queries:
* Rednecks
* Troll Trace
* Respect my authority
* Respect my authoritah
* Respected my authority

In [29]:
# build the index here
# add cells as needed to organize your code
mat={}
num=len(documents)
for i in xrange(num):
    doc=documents[i];
    for word in doc:
        if word in mat:
            mat[word][i]=True;
        else:
            mat[word]=[False]*num;
            mat[word][i]=True;

In [ ]:
import numpy as np
import re
from nltk import stem
stemmer = stem.PorterStemmer()

search_text = raw_input('Boolean Search:')
# search for the input using your index and print out ids of matching documents
#search=re.split(";\W*|,\W*|\*\W*|\n\W*|\s+|-\W*|\.\W*|\:\W*|\?\W*|!\W*|_\W*|\'\W*",search_text)
if search_text:
    search=re.split("\W+",search_text);
    size=len(search)
    for i in xrange(size):
        word=search[i]
        if use_stemming and use_casefolding:
            search[i]=stemmer.stem(word.lower());
        if not use_stemming and use_casefolding:
            search[i]=word.lower();
        if use_stemming and not use_casefolding:
            search[i]=stemmer.stem(word);
    word=search[0];
    rs=set();
    if word in mat:
        bool_array=np.array(mat[word])
        indexes=np.where(bool_array==True)[0];
        for idx in indexes:
            rs.add(idx);
        for idx in xrange(size):
            word=search[idx];
            if not word in mat:
                rs.clear();
                break;
            bool_list=mat[word];
            set_list=list(rs);
            for ids in set_list:
                if not bool_list[ids]:
                    rs.remove(ids);
            if len(rs)==0:
                break;
    for idx in rs:
        print names[idx]

### Observations

When we use stemming and casefolding, is the result different from the result when we do not use them? Do you find cases where you prefer stemming? Or not? Or cases where you prefer casefolding? Or Not? Write down your observations below.

The results are often different. When I want to look up for words with no concern of the grammer and just for the meanning itself, it's better to use stemming, but when we want to be more specific, we don't need then, such as, when we want to find 'bikes', use stemming, we will find 'bike' and 'bikes', which is a kind of interuption. As for casefolding, if we care about the big letter and small letter, it will make trouble, for example, when we want 'china', we will find 'China' as well, however, if we do not use it, for cases that words are at the beginning of the sentence, we will lose some results.

### BONUS: Phrase Queries

Your search engine needs to also (optionally) support phrase queries of arbitrary length. Use quotes in a query to tell your search engine this is a phrase query. Again, we don't explicitly type AND in queries and never use OR, NOT, or parentheses.

In [ ]:
# Your code here

In [ ]:
search_text = raw_input('Boolean Search (Phrase Query:')
# search for the input using your index and print out ids of matching documents

## (40 points) Part 3: Ranked Retrieval

In this part, your job is to support queries over an index that you build. This time you will use the vector space model plus cosine similarity to rank documents.

**TFIDF:** For the document vectors, use the standard TFIDF scores. That is, use the log-weighted term frequency $1+log(tf)$; and the log-weighted inverse document frequency $log(\frac{N}{df})$. For the query vector, use simple weights (the raw term frequency). For example:
* query: troll $\rightarrow$ (1)
* query: troll trace $\rightarrow$ (1, 1)

**Output:**
For a given query, you should rank all the 277 documents but you only need to output the top-5 documents (i.e. document ids) plus the cosine score of each of these documents. For example:

* result1 - score1
* result2 - score2
* result3 - score3
* result4 - score4
* result5 - score5

You can additionally assume that your queries will contain at most three words. Be sure to normalize your vectors as part of the cosine calculation!

In [30]:
# build the vector space index here
# add cells as needed to organize your code
import numpy as np
import math
mat={}
num=len(documents)
for i in xrange(num):
    doc=documents[i];
    for word in doc:
        if word in mat:
            mat[word][i]+=1.;
        else:
            mat[word]=[0.]*num;
            mat[word][i]=1.;

docf={};
for word in mat:
    count=0.;
    for k in mat[word]:
        if k!=0.:
             count+=1.;
    if count!=0.:
        docf[word]=math.log10(num*1.0/count);
    else:
        docf[word]=0.;
        
for word in mat:
    for i in xrange(num):
        if mat[word][i]!=0.:
            mat[word][i]=docf[word]*(1.+math.log10(mat[word][i]));

In [ ]:
import numpy as np
import heapq as hp
from nltk import stem
stemmer = stem.PorterStemmer()
import re
import math

now_size=0;
search_text = raw_input('Ranked Search:')
# search for the input and print the top 5 document ids along with their associated cosine scores.
if search_text:
    search=re.split("\W+",search_text);
    h=[];
    size=len(search)
    filtered=[];
    for i in xrange(size):
        word=search[i]
        if use_stemming and use_casefolding and stemmer.stem(word.lower()) in mat:
            filtered.append(stemmer.stem(word.lower()));
        if not use_stemming and use_casefolding and word.lower() in mat:
            filtered.append(word.lower());
        if use_stemming and not use_casefolding and stemmer.stem(word) in mat:
            filtered.append(stemmer.stem(word));
    now_size=len(filtered)
    
if now_size>0:
    num=len(documents)
    num_w=len(mat.keys())
    v={};
    for word in mat:
        v[word]=0.;
    for word in filtered:
        v[word]+=1.;
    v1=np.array([0.]*num_w);
    idx=0;
    for word in mat:
        v1[idx]=v[word];
        idx+=1;
    x=math.sqrt(np.sum(v1*v1));
    v1/=x;
                
    for i in xrange(num):
        score=np.array([0.]*num_w);
        idx=0;
        for word in mat:
            score[idx]=mat[word][i];
            idx+=1;
        le=math.sqrt(np.sum(score*score));
        if le==0.:
            hp.heappush(h,(0.,names[i]));
        else:
            fina=np.sum(v1*score)/le;
            #print fina
            hp.heappush(h,[fina,names[i]]);
        if len(h)>5:
            hp.heappop(h);

    while len(h)>0:
        entry=hp.heappop(h);
        print entry[1],'-',entry[0] ;

## How we grade

The grader will randomly pick 5-10 queries to test your program. You are welcome to discuss the results returned by your search engine with others on Piazza.